In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import bbknn
import os
from scipy import sparse
import matplotlib.pyplot as plt
# from scanpy_base_moudle_update2 import *
# import scrublet as scr
import datetime
import harmonypy as hm

sc.settings.verbosity = 3
#sc.logging.print_versions()
# 设置图片的分辨率以及其他样式
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=12)

import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        #'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

In [ ]:
adata = sc.read('/mnt/data/Project2021/Gut_immune_surveillance/data/h5ad/All_579886_umap.h5ad')
adata

In [ ]:
from matplotlib import cm, colors

sc.settings.set_figure_params(dpi=400, figsize = (4, 3), fontsize=12)

colors2 = plt.cm.jet(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

sc.pl.umap(adata, color=['DEFA5','DEFA6','ITLN2','PLA2G2A'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['REG3A','PRSS2','REG3G','LYZ'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['NR1H4','NR1H3','VDR','NR1I3'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)

# Immune cells

In [ ]:
adata.obs['Compartment'].cat.categories

In [ ]:
obs_index = adata.obs.loc[adata.obs["Compartment"].isin(['T/ILCs','B cells','MNPs','Mast cells']), :].index
adata = adata[obs_index, :]
adata

In [ ]:
sc.pl.umap(adata, color = ['Compartment'])

In [ ]:
def progress_pca_bbknn_umap_tsne(adata, n_pcs=20, batch_key='bbknn_batch'):
    ## 主成分分析降维
    sc.tl.pca(adata, svd_solver='arpack')
    # 在PCA坐标中绘制散点图
    sc.pl.pca(adata)
    # 可视化每个PC对数据总方差的贡献, 这为我们提供了有关为计算单元的邻域关系应考虑的PC数量的信息sc.tl.tsne()
    sc.pl.pca_variance_ratio(adata, log=True)

    # 使用bbknn算法处理批次效应(代替了neighbors)
    # bbknn要求数据已经过标准化和主成分分析
    starttime = datetime.datetime.now()
    adata.obs['bbknn_batch'] = [str(i)+'-'+str(j) for i,j in zip(adata.obs['dataset'], adata.obs['batch_name'])]
    sc.external.pp.bbknn(adata, batch_key=batch_key, n_pcs=n_pcs, neighbors_within_batch=1)
    endtime = datetime.datetime.now()
    print((endtime-starttime).seconds)
    
    # 计算UMAP
    sc.tl.umap(adata)
    # sc.tl.tsne(adata, n_jobs=6)

    return adata

adata = progress_pca_bbknn_umap_tsne(adata)

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 3), fontsize=10)
sc.pl.umap(adata, color=['Compartment'], add_outline=True,
           palette=sc.pl.palettes.vega_20_scanpy, outline_width = (0.2, 0.05), frameon=False)

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 3), fontsize=10)
sc.pl.umap(adata, color=['cluster_figure1A'], add_outline=True,
           palette="tab20_r", outline_width = (0.2, 0.05), frameon=False)

In [ ]:
old_colors = np.array(adata.uns['cluster_figure1A_colors'])
new_colors = old_colors

In [ ]:
## T/ILCs

# NK/ILC1
new_colors[[0]] = '#ff19e0'
# ILC2/3
new_colors[[1]] = '#f6ff00'
# gamma_delta T
new_colors[[2]] = '#ff0000'
# CD4+Naive T/Treg/Tfh
new_colors[[3,4,5]] = '#9edae5'
# CD4+T helper
new_colors[[6,7,8,9]] = '#17becf'
# CD8+Naive T/ Cytotoxic T /DEFA5+/MT1X+
new_colors[[10,11,12,13,14,15]] = '#4265bd'
# Cycling T
new_colors[[16]] = '#00f3f7'#fff7b0

## B cells

# IgA Plasma
new_colors[[17]] = '#f7a85b'
# IgG Plasma
new_colors[[18]] = '#ff0505'
# Cycling Plamsa cells
new_colors[[19]] = '#00f3f7'
# GC B cells/Cycling GC B cells
new_colors[[21,22]] = '#00f3f7'
# Naive B cells
new_colors[[20]] = '#fff70a'
# Memory B cells/FCRL4+Memory B cells
new_colors[[23,24]] = '#ffa90a'

## DCs and MNPs

# Monocytes
new_colors[[25,26,27,28]] = '#ff9896'
# Megakaryocytes
new_colors[[29]] = '#fff70a'
# DCs
new_colors[[30,31,32,33]] = '#d62728'
# M1 Macrophages
new_colors[[34,35]] = '#98df8a'
# M2 Macrophages
new_colors[[36,37,38,39]] = '#2ca02c'
# Other Macrophages
new_colors[[40,41,42,43,44]] = '#008700'
# Cycling+MNPs
new_colors[[45]] = '#00f3f7'

## Mast 'C05-Mast cells'
new_colors[[46]] = '#9c1a43'

In [ ]:
# 论文里的图片
adata.uns['cluster_figure1A_colors'] = new_colors

# Figure 1a 按细胞类型划分颜色
sc.settings.set_figure_params(dpi=400, figsize = (4, 3), fontsize=7)
sc.pl.umap(adata, color=['cluster_figure1A'], add_outline=True, frameon=False, title='')

In [ ]:
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=7)
sc.pl.umap(adata, color=['cluster_figure1A'], groups = ['C03-B05-GC B cells'] ,add_outline=True, frameon=False)

In [ ]:
from matplotlib import cm, colors
sc.settings.set_figure_params(dpi=400, figsize = (4, 3), fontsize=7)

colors2 = plt.cm.plasma(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,10))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

sc.pl.umap(adata, color=['DEFA5','DEFA6','PRSS2'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,vmin=0,vmax=6)
sc.pl.umap(adata, color=['REG3A','ITLN2','PLA2G2A'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,vmin=0,vmax=6)
sc.pl.umap(adata, color=['CAMP','FCRL4','CD79A'], add_outline=False, frameon=False,size = 1.8, color_map = mymap)
sc.pl.umap(adata, color=['FPR2','P2RX7'], add_outline=False, frameon=False,size = 1.8, color_map = mymap)

In [ ]:
sc.pl.umap(adata, color=['CAMP','FCRL3','FCRL5'], add_outline=False, frameon=False,size = 1.8, color_map = mymap)

In [ ]:
adata.obs['cluster_figure1A'].cat.categories

In [ ]:
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=7)
sc.pl.umap(adata, color=['cluster_figure1A'], groups = ['C02-T01-NKs and ILC1'] ,add_outline=True, frameon=False)

In [ ]:
# NK
sc.pl.umap(adata, color=['GNLY','GZMA','GZMB','GZMK'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['IFNG','PRF1','NKG7','EOMES'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
# ILC1
sc.pl.umap(adata, color=['CCL3','CCL4','CCL5','FCRL3'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['XCL1','XCL1','FCRL1','FCRL2'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)

In [ ]:
adata.write('/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/h5ad/Immune_cells.h5ad')

In [ ]:
adata = sc.read('/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/h5ad/Immune_cells.h5ad')

**计算DEFA5+cells占各区室各器官的比例**

In [ ]:
adata

In [ ]:
obs_index = adata.obs.loc[adata.obs["health"].isin(['Normal']), :].index
adata = adata[obs_index, :]
adata

In [ ]:
obs_index = adata.obs.loc[~adata.obs["organ"].isin(['Appendix']), :].index
adata = adata[obs_index, :]
adata

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['cluster_figure1A'],  # Make a crosstab
                columns=adata.obs['organ'], margins=True)  # Name the count column
Groups_tab_1

In [ ]:
adata_raw = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)
adata_raw

In [ ]:
DEFA5_genes_index = adata.var_names.isin(['DEFA5'])

adata.obs['count_DEFA5'] = np.sum(adata[:, DEFA5_genes_index].X, axis=1)

# 获取每个细胞的得分
compartments_values_list_except = adata.obs.loc[:,['count_DEFA5']].values.tolist()
cell_compartments_list = []

for i in compartments_values_list_except:

    if ((i[0]>0.1)):
        j = 'DEFA5+'   
    else:
        j = 'DEFA5-'
        
    cell_compartments_list.append(j)
    
adata.obs['Compartments'] = cell_compartments_list
adata.obs['Compartments']

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['Compartment'],  # Make a crosstab
                columns=adata.obs['Compartments'], margins=True)  # Name the count column
Groups_tab_1

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['Compartment'],  # Make a crosstab
                columns=adata.obs['pan_organ'], margins=True)  # Name the count column
Groups_tab_1

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['Compartment'],  # Make a crosstab
                columns=adata.obs['organ'], margins=True)  # Name the count column
Groups_tab_1

In [ ]:
obs_index = adata.obs.loc[adata.obs["pan_organ"].isin(['Duo-Jej']), :].index
adata_pro = adata[obs_index, :]
adata_pro

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata_pro.obs['Compartment'],  # Make a crosstab
                columns=adata_pro.obs['Compartments'], margins=True)  # Name the count column
Groups_tab_1

# 0.0964, 0.0865, 0.0337, 0.1663,

In [ ]:
obs_index = adata.obs.loc[adata.obs["pan_organ"].isin(['Ileum']), :].index
adata_ile = adata[obs_index, :]
adata_ile

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata_ile.obs['Compartment'],  # Make a crosstab
                columns=adata_ile.obs['Compartments'], margins=True)  # Name the count column
Groups_tab_1
# 0.1886, 0.3811, 0.1538, 0.2634,

In [ ]:
obs_index = adata.obs.loc[adata.obs["pan_organ"].isin(['App-Col-Rec']), :].index
adata_LI = adata[obs_index, :]
adata_LI

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata_LI.obs['Compartment'],  # Make a crosstab
                columns=adata_LI.obs['Compartments'], margins=True)  # Name the count column
Groups_tab_1
# 0.0005, 0, 0, 0

In [ ]:
sc.settings.set_figure_params(dpi=300, fontsize=17)
# create a dataset
height = [0.1663, 0.0964, 0.0865, 0.0337,0, 0.2634, 0.1886, 0.3811, 0.1538, 0, 0, 0.0005, 0, 0] # Duo-Stem, Paneth, Jej-Stem, Paneth, Ile-Stem, Paneth
bars = (' ', ' ', ' ',' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
x_pos = np.arange(len(bars))

fig, ax = plt.subplots(figsize=(8, 4))
# Create bars with different colors
plt.bar(x_pos, height, color=['#427637', '#cc2028', '#224de6', '#f2a51f',
                              'green', 
                              '#427637', '#cc2028', '#224de6', '#f2a51f',
                              'green',
                              '#427637', '#cc2028', '#224de6', '#f2a51f',],
            edgecolor = '#000000',
            linewidth=0.4,
            width=0.8,)

# Create names on the x-axis
plt.xticks(x_pos, bars, fontsize = 30)
plt.xticks([])
plt.yticks([0.0, 0.100, 0.200, 0.300, 0.400], fontsize = 19)

plt.grid(False)

print(ax.axis())
# ax.axis([-0.54, 2.5400000000000005, 0.0, 1])

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
help(plt.bar)

# T and ILCs

In [ ]:
adata = sc.read('/mnt/data/Project2021/Gut_immune_surveillance/data/h5ad/T_ILCs_after_born_harmony_annoation.h5ad')
adata

In [ ]:
tf_GENES = '/mnt/data/project/scenic/auxilliaries/lambert2018_c.txt'
amps_pd = pd.read_table(tf_GENES)
amps_pd.head(5)

In [ ]:
# 141
amps_list = list(amps_pd['Gene_name'])
amps_list = [x for x in amps_list if x in adata.raw.var_names]
len(amps_list)

In [ ]:
adata_raw = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)
adata_c = adata_raw[:,amps_list]
adata_c

In [ ]:
sc.tl.rank_genes_groups(adata_c, 'cluster_figure1A', method='wilcoxon')

In [ ]:
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=20)
sc.pl.rank_genes_groups_dotplot(adata_c, 
                                n_genes=4,
                                use_raw=False,
                                values_to_plot='logfoldchanges', 
                                min_logfoldchange=0.5, 
                                vmax=3, 
                                vmin=-3, 
                                cmap='bwr')

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 3), fontsize=10)
sc.pl.umap(adata, color=['cluster_figure1A'], add_outline=True,
           palette=sc.pl.palettes.vega_20_scanpy, outline_width = (0.2, 0.05), frameon=False, title='')

sc.pl.umap(adata, color=['cluster_figure1A'], add_outline=True, outline_width = (0.2, 0.05), 
           palette=sc.pl.palettes.vega_20_scanpy, frameon=False, title='', size=0.5)

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 3), fontsize=20)
# sc.tl.rank_genes_groups(adata, 'cluster_figure1A', method='wilcoxon')
sc.pl.rank_genes_groups_dotplot(adata, 
                                n_genes=8,
                                use_raw=True,
                                values_to_plot='logfoldchanges', 
                                min_logfoldchange=1, 
                                vmax=3, 
                                vmin=-3, 
                                cmap='bwr')

In [ ]:
marker_genes_dict = {'Pan-marker': ['PTPRC','CD3D','CD3E'],
    'CD4/CD8': ['CD4','CD8A','CD8B'],
    'non_alpha_beta T': ['TYROBP','FCER1G','TRDC'],
    'NK and ILC1': ['GNLY','GZMA','GZMB','GZMK','IFNG','PRF1','NKG7','EOMES','CCL3','CCL4','CCL5','XCL1','XCL1'],
    'ILC2 and 3': ['AREG','DLL1','IL2RA','KIT','IL7R','IL22','IL23R','CCR6','LTB','LST1','IL4I1','CCL20'],
    'gamma_delta T': ['GFI1','MYBL1','TRDC','CCL5','ITGA1','TRAC','ENTPD1','ABI3','CD247','CD7','CDH17'],                     
    'Naive/Tcm': ['TCF7','LEF1','RBAK','CCR7','KLF2','SELL','NOSIP','CD27'], 
    'CD4+Treg': ['FOXP3','BATF','TIGIT','CTLA4','TNFRSF4','TNFRSF18','ICA1'],
    'CD4+Type 1 cytokines Trm': ['IFNG','IL2','TNF','CXCR3','CCR9','CCL5'],
    'CD4+Type 3 cytokines Trm': ['IL17A','IL17F','IL22','CCR6','DPP4','IL23R','CCL20'], 
    'CD8+Cytotoxic Trm': ['IL7R','KLRB1','S100A4','MGAT4A','FKBP11','CD9','SPINK2'],
    'CD8+CTL Tem': ['GZMK','GZMB','GZMH','KLRG1','NKG7','CD44','CCL3','CCL4','CCL5','CST7'],
    'CD8+IEL': ['KLRD1','CD7','CCL5','HOPX','ENTPD1','KLRC1','KLRC2','KLRC3','NR4A1','NR4A2','NR4A3'],
}
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=18)
mp = sc.pl.dotplot(adata, 
              marker_genes_dict, 
              'cluster_figure1A', 
              dendrogram=False, 
              #figsize=(3, 4),
              use_raw=True,
              cmap = 'Reds',
              var_group_rotation=45,
              #swap_axes=True,
              standard_scale='var',
              colorbar_title=None,
              return_fig=True
              # , save='_' + sample_name + '_fig11.png'
             ) 
mp.style(grid=True,cmap = 'Reds').show() # RdYlBu_r

In [ ]:
list(adata.obs['cluster_figure1A'].cat.categories)

In [ ]:
markers = ['CD3D','CD4','CD8A', # pan-T
           #'TYROBP','TRDC', # non-alphabelta-T
           'GNLY','CCL3', # NK-ILC1
           'AREG','KIT',  # ILC2-3
           'ENTPD1','ABI3',# gammadelta-T
           'SELL','CCR7', # Naive
           'FOXP3','BATF', # Treg
           'PASK','CXCR3',# Tfh
           'IL2','TNF', # Th1
           'IL17A','IL22',# Th17
           'ANXA1','RGCC', # CD4+ANXA1+T cells
           'KLRD1','CD9', # CD8+
           'GZMK','GZMB', # CD8+CTL
           'KLRC2','KLRC3', # CD8+IEL
           'DEFA5','DEFA6','REG3A','PLA2G2A',
           'MT1X','MT1E',
           'MKI67','UBE2C'
          
          ]
categories_order = list(adata.obs['cluster_figure1A'].cat.categories)

In [ ]:
    mp = sc.pl.matrixplot(adata, 
                     markers, 
                     groupby='cluster_figure1A', 
                     dendrogram=False, 
                     return_fig=True,
                     cmap='RdYlBu_r',
                     categories_order = categories_order,
                     standard_scale='var'
                    )
    mp.style(edge_color='white',cmap = 'RdYlBu_r',edge_lw=0.7).show()

In [ ]:
markers = ['CD3D','CD4','CD8A', # pan-T
           #'TYROBP','TRDC', # non-alphabelta-T
           'CCL3', # NK-ILC1
           'KIT',  # ILC2-3
           'ENTPD1',# gammadelta-T
           'CCR7', # CD4+Naive
           'FOXP3', # Treg
           'PASK',# Tfh
           'IL2', # Th1
           'IL17A',# Th17
           'ANXA1',# CD4+ANXA1+T cells
           'SELL', # CD8+Naive
           'CD9', # CD8+
           'GZMK',# CD8+CTL
           'KLRC3', # CD8+IEL
           'DEFA5','DEFA6','REG3A','PLA2G2A',
           'MT1X',
           'MKI67'
          
          ]
categories_order = list(adata.obs['cluster_figure1A'].cat.categories)

mp = sc.pl.matrixplot(adata, 
                     markers, 
                     groupby='cluster_figure1A', 
                     dendrogram=False, 
                     return_fig=True,
                     cmap='RdYlBu_r',
                     categories_order = categories_order,
                     standard_scale='var'
                    )
mp.style(edge_color='white',cmap = 'RdYlBu_r',edge_lw=0.7).show()

In [ ]:
sc.settings.set_figure_params(dpi=300,fontsize=18)
mp = sc.pl.dotplot(adata, 
              markers, 
              'cluster_figure1A', 
              dendrogram=False, 
              #figsize=(3, 4),
              use_raw=True,
              cmap = 'Reds',
              var_group_rotation=45,
              #swap_axes=True,
              categories_order = categories_order,
              standard_scale='var',
              colorbar_title=None,
              return_fig=True
              # , save='_' + sample_name + '_fig11.png'
             ) 
mp.style(grid=True,cmap = 'Spectral_r').show() # RdYlBu_r

In [ ]:
from matplotlib import cm, colors

sc.settings.set_figure_params(dpi=400, figsize = (4, 3), fontsize=12)

colors2 = plt.cm.jet(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

sc.pl.umap(adata, color=['DEFA5','DEFA6','REG3A','PLA2G2A'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['ITLN2','PRSS2','REG3G','LYZ'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['NR1H4','NR1H3','VDR','NR1I3'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)

In [ ]:
sc.pl.umap(adata, color=['CD4','CD8A'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)

In [ ]:
plt.cm

In [ ]:
colors2 = plt.cm.Greys_r(np.linspace(0.7,0.8, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,30))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['EPCAM','KRT8'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)

In [ ]:
colors2 = plt.cm.jet(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,30))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['CD3D','KRT18','KRT19'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)

In [ ]:
obs_index = adata.obs.loc[adata.obs["health"].isin(['Normal']), :].index
adata = adata[obs_index, :]
adata

In [ ]:
obs_index = adata.obs.loc[~adata.obs["organ"].isin(['Appendix']), :].index
adata = adata[obs_index, :]
adata

In [ ]:
adata_raw = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)
adata_raw

In [ ]:
# 修改器官名，以使绘制bar图时按正确顺序排列。
organ_list = []

for i in adata.obs['pan_organ'].to_list():
    
    if i == 'Duo-Jej':
        j = '1-Pro_SI'
    elif i == 'Ileum':
        j = '2-Ileum'
    elif i == 'App-Col-Rec':
        j = '3-LI'        
    organ_list.append(j)

adata.obs['organ_name'] = organ_list

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['organ_name'],  # Make a crosstab
                        columns=adata.obs['cluster_figure1A'], margins=True)               # Name the count column
MyTab_1= Groups_tab_1.div(Groups_tab_1["All"], axis=0)
MyTab2_1 = MyTab_1.drop(columns="All")
MyTab2_1 = MyTab2_1.drop(index="All")
MyTab2_1.T

In [ ]:
            color={"C01-E01-Mouse-Stem cells": "#1f76b4",
                   "C01-E02-Mouse-TA": "#2bc6ed",
                   "C01-E03-Mouse-Immature colonocytes": "#279e68", 
                   "C01-E03-Mouse-Immature enterocytes": "#279e68", 
                   "C01-E04-Mouse-APOA1+Enterocytes": "#d62727",
                   "C01-E04-Mouse-Aqp8+colonocytes": "#ab40fc", 
                   "C01-E05-Mouse-Fcgbp+colonocytes": "#fc40f6", 
                   "C01-E09-Mouse-Microfold cells": "#17bdcf",                  
                   "C01-E10-Mouse-Enteroendocrine cells": "#aec7e8", 
                   "C01-E11-Mouse-Tuft cells": "#ffbc78", 
                   "C01-E12-Mouse-Goblet cells": "#98df8a",                   
                   "C01-E12-Mouse-Immature goblet cells": "#d3fac5", 
                   "C01-E13-Mouse-Goblet": "#98df8a", 
                   "C01-E13-Mouse-Paneth cells": "#ff9896",                   }

In [ ]:
#categories = IMM_group[::-1]

MyTab2_1.columns = pd.CategoricalIndex(MyTab2_1.columns.values)

# Sort the columns (axis=1) by the new categorical ordering
MyTab2_1 = MyTab2_1.sort_index(axis=1)

ax = MyTab2_1.plot.bar(
            figsize=(2,5),
            stacked=True,
            edgecolor = '#000000',
            linewidth=0.4,
            width=0.8, 
            fontsize=15,

             )


plt.title("", fontsize=12)
plt.ylabel("Fraction of cells", fontsize=14)
plt.xlabel("", fontsize=12)
plt.ylim=1.0

#plt.gca().get_legend().remove() #remove legend
# plt.legend(categories, loc='center left', bbox_to_anchor=(1, 0.6), fontsize=12)
# plt.savefig('Proportion of clusters accross organs.png')
# 去除刻度
#plt.xticks([])
#plt.yticks([])
# ax.tick_params(bottom=False, top=False, left=False, right=False)
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='center left', bbox_to_anchor=(1, 0.6), fontsize=8)
plt.grid(False)

plt.show()

In [ ]:
MyTab2_EPI

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['cluster_figure1A'],  # Make a crosstab
                        columns=adata.obs['organ_name'], margins=True)               # Name the count column

MyTab_1= Groups_tab_1.div(Groups_tab_1.iloc[-1], axis=1)
MyTab2_1 = MyTab_1.drop(columns="All")
MyTab2_1 = MyTab2_1.drop(index="All")
MyTab2_1['All'] = MyTab2_1.apply(lambda x: x.sum(), axis=1)
MyTab2_1= MyTab2_1.div(MyTab2_1["All"], axis=0)
MyTab2_EPI = MyTab2_1.drop(columns="All")
MyTab2_EPI

categories = ['3-LI','2-Ileum','1-Pro_SI']

MyTab2_EPI.columns = pd.CategoricalIndex(MyTab2_EPI.columns.values, 
                                 ordered=True, 
                                 categories=categories)

# Sort the columns (axis=1) by the new categorical ordering
MyTab2_EPI = MyTab2_EPI.sort_index(axis=1)

ax = MyTab2_EPI.plot.bar(
            figsize=(10,6),
            stacked=True, 
            edgecolor = '#000000',
            linewidth=0.5,
            width=0.8, 
            fontsize=15,
            color={"1-Pro_SI": "#393b79","2-Ileum": "#8ca252","3-LI": "#e7ba52"}
             )


plt.title("", fontsize=12)
plt.ylabel("Fraction of cells", fontsize=15)
plt.xlabel("", fontsize=15)
plt.ylim=1.0

#plt.gca().get_legend().remove() #remove legend
# plt.legend(categories, loc='center left', bbox_to_anchor=(1, 0.6), fontsize=12)
# plt.savefig('Proportion of clusters accross organs.png')
# 去除刻度
plt.xticks([])
plt.yticks([])
# ax.tick_params(bottom=False, top=False, left=False, right=False)
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='center left', bbox_to_anchor=(1, 0.6), fontsize=12)

plt.show()

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['cluster_figure1A'],  # Make a crosstab
                        columns=adata.obs['organ_name'], margins=True)               # Name the count column

MyTab_1= Groups_tab_1.div(Groups_tab_1.iloc[-1], axis=1)
MyTab2_1 = MyTab_1.drop(columns="All")
MyTab2_1 = MyTab2_1.drop(index="All")
MyTab2_1['All'] = MyTab2_1.apply(lambda x: x.sum(), axis=1)
MyTab2_1= MyTab2_1.div(MyTab2_1["All"], axis=0)
MyTab2_EPI = MyTab2_1.drop(columns="All")
MyTab2_EPI

categories = ['3-LI','2-Ileum','1-Pro_SI']

MyTab2_EPI.columns = pd.CategoricalIndex(MyTab2_EPI.columns.values, 
                                 ordered=True, 
                                 categories=categories)

# Sort the columns (axis=1) by the new categorical ordering
MyTab2_EPI = MyTab2_EPI.sort_index(axis=1)

ax = MyTab2_EPI.plot.bar(
            figsize=(10,6),
            stacked=True, 
            edgecolor = '#000000',
            linewidth=0.5,
            width=0.8, 
            fontsize=15,
            # color={"1-Pro_SI": "#393b79","2-Ileum": "#8ca252","3-LI": "#e7ba52"}
             )


plt.title("", fontsize=12)
plt.ylabel("Fraction of cells", fontsize=15)
plt.xlabel("", fontsize=15)
plt.ylim=1.0

#plt.gca().get_legend().remove() #remove legend
# plt.legend(categories, loc='center left', bbox_to_anchor=(1, 0.6), fontsize=12)
# plt.savefig('Proportion of clusters accross organs.png')
# 去除刻度
# plt.xticks([])
#plt.yticks([])
# ax.tick_params(bottom=False, top=False, left=False, right=False)
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='center left', bbox_to_anchor=(1, 0.6), fontsize=12)

plt.show()

In [ ]:
markers = ['DEFA5','DEFA6','PLA2G2A','ITLN2','REG3G','REG3A','NR1H4','NR1H3','VDR']
categories_order = ['Duodenum','Jejunum','Ileum','Colon']

In [ ]:
    mp = sc.pl.matrixplot(adata, 
                     markers, 
                     groupby='organ', 
                     dendrogram=False, 
                     return_fig=True,
                     cmap='bwr',
                     categories_order = categories_order,
                     standard_scale='var'
                    )
    mp.style(edge_color='white',cmap = 'RdYlBu_r',edge_lw=1.0).show()

In [ ]:
adata = sc.read('/mnt/data/Project2021/Gut_immune_surveillance/data/h5ad/T_ILCs_after_born_harmony_annoation.h5ad')
obs_index = adata.obs.loc[adata.obs["organ"].isin(['Ileum']), :].index
adata = adata[obs_index, :]
adata

In [ ]:
adata = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)
adata

In [ ]:
adata_pre_matrix = adata.to_df()
adata_pre_matrix.head()

In [ ]:
adata_pre_matrix_1 = adata_pre_matrix.loc[:,['DEFA5','DEFA6','REG3A','PLA2G2A','NR1H4','NR1H3','VDR']]
adata_pre_matrix_1["disease_state"] = adata.obs["disease_state"]

adata_pre_matrix_1 = pd.melt(adata_pre_matrix_1, value_vars=['DEFA5','DEFA6','REG3A','PLA2G2A','NR1H4','NR1H3','VDR'],id_vars = ['disease_state'],
             var_name='AMPs', value_name='log2(TP10k+1)')

adata_pre_matrix_1

In [ ]:
import seaborn as sns

sns.set(style="ticks", 
        font='Arial', 
        font_scale=1.0, 
        rc={'figure.figsize':(3,6)}
       )

ax = sns.pointplot(x="disease_state", y="log2(TP10k+1)", hue="AMPs",
                   data=adata_pre_matrix_1,
                   markers=["o", "o", "o", "o", "s", "s", "s"],
                   linestyles=["-", "-", "-", "-", "-", "-", "-", "-", "-"],
                   palette = ["#ff0000", "#ff00c3", "#ff4d00", "#ffd000","#00c3ff","#001aff","#00b359"],
                   order = ["Healthy","Uninflamed","Inflamed"],
                   capsize=.1,     # 误差帽的粗细
                   errwidth = 1.5,  # 误差棒的粗细
                   scale = 1.0     # 连线的簇细，默认值为1
                  )     

## 删除坐标轴标记
ax.set(ylabel="")
ax.set(xlabel="")

# 移除图例
# plt.legend([],[], frameon=False)

# 删除坐标轴上线和右线，当Trim为真时，删除数据数值范围外的坐标轴
sns.despine() # trim=True

print(ax.axis())

# ax.axis([-0.2, 2.2, 0.023832513266243038, 0.6])

In [ ]:
adata_pre_matrix_1 = adata_pre_matrix.loc[:,['DEFA5','DEFA6','REG3A','PLA2G2A']]
adata_pre_matrix_1["disease_state"] = adata.obs["disease_state"]

adata_pre_matrix_1 = pd.melt(adata_pre_matrix_1, value_vars=['DEFA5','DEFA6','REG3A','PLA2G2A'],id_vars = ['disease_state'],
             var_name='AMPs', value_name='log2(TP10k+1)')

import seaborn as sns

sns.set(style="ticks", 
        font='Arial', 
        font_scale=1.0, 
        rc={'figure.figsize':(3,6)}
       )

ax = sns.pointplot(x="disease_state", y="log2(TP10k+1)", hue="AMPs",
                   data=adata_pre_matrix_1,
                   markers=["o", "o", "o", "o"],
                   linestyles=["-", "-", "-", "-", "-"],
                   palette = ["#ff0000", "#088722", "#001aff", "#ffd000"],
                   order = ["Healthy","Uninflamed","Inflamed"],
                   capsize=.1,     # 误差帽的粗细
                   errwidth = 1.5,  # 误差棒的粗细
                   scale = 1.0     # 连线的簇细，默认值为1
                  )     

## 删除坐标轴标记
ax.set(ylabel="")
ax.set(xlabel="")

# 移除图例
# plt.legend([],[], frameon=False)

# 删除坐标轴上线和右线，当Trim为真时，删除数据数值范围外的坐标轴
sns.despine() # trim=True

print(ax.axis())

# ax.axis([-0.2, 2.2, 0.023832513266243038, 0.6])

In [ ]:
adata_pre_matrix_1 = adata_pre_matrix.loc[:,['DEFA5','DEFA6','REG3A','PLA2G2A']]
adata_pre_matrix_1["disease_state"] = adata.obs["disease_state"]

adata_pre_matrix_1 = pd.melt(adata_pre_matrix_1, value_vars=['DEFA5','DEFA6','REG3A','PLA2G2A'],id_vars = ['disease_state'],
             var_name='AMPs', value_name='log2(TP10k+1)')

import seaborn as sns

sns.set(style="ticks", 
        font='Arial', 
        font_scale=1.5, 
        rc={'figure.figsize':(4.5,6)}
       )

ax = sns.pointplot(x="disease_state", y="log2(TP10k+1)", hue="AMPs",
                   data=adata_pre_matrix_1,
                   markers=["o", "o", "o", "o"],
                   linestyles=["-", "-", "-", "-", "-"],
                   palette = ["#ff0000", "#088722", "#001aff", "#ffd000"],
                   order = ["Healthy","Uninflamed","Inflamed"],
                   capsize=.1,     # 误差帽的粗细
                   errwidth = 1.5,  # 误差棒的粗细
                   scale = 1.0     # 连线的簇细，默认值为1
                  )     

## 删除坐标轴标记
ax.set(ylabel="")
ax.set(xlabel="")

# 移除图例
plt.legend([],[], frameon=False)

# 删除坐标轴上线和右线，当Trim为真时，删除数据数值范围外的坐标轴
sns.despine() # trim=True

print(ax.axis())

# ax.axis([-0.2, 2.2, 0.023832513266243038, 0.6])

In [ ]:
from statannot import add_stat_annotation

def stat_boxplot(df, x, y, box_pairs, order=None, palette=None):
    
    ## 绘图函数
    sns.set(style="ticks", font='Arial', font_scale=1.0, rc={'figure.figsize':(5,2)})
    ax = sns.boxplot(data=df, x=x, y=y, order=order, palette=palette)

    ## 显著性计算和可视化函数
    add_stat_annotation(ax, plot='boxplot', data=df, x=x, y=y, order=order, 
                        box_pairs=box_pairs, test='t-test_ind', loc='inside', verbose=2)

    ## 坐标轴设置
    ax.set(ylabel=y)
    ax.set(xlabel="")
    sns.despine(trim=True, left=True)
    plt.grid(False)
    plt.show()

In [ ]:
adata = sc.read('/mnt/data/Project2021/Gut_immune_surveillance/data/h5ad/T_ILCs_after_born_harmony_annoation.h5ad')
obs_index = adata.obs.loc[adata.obs["organ"].isin(['Ileum']), :].index
adata = adata[obs_index, :]

adata = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)

adata_raw_matrix = adata.to_df()
adata_raw_matrix.head()

In [ ]:
adata_raw_matrix['disease_state'] = adata.obs['disease_state']

In [ ]:
AMPs_list = 'DEFA5 DEFA6 REG3A REG3G PRSS2 GBP1 PLA2G2A LYZ ANG ADM DMBT1 WFDC2 LYPD8 ITLN1 ITLN2 SLPI PI3 DEFB1 RNASE6 CAMP LEAP2 NR1H4 NR1H3 VDR TCF4 TCF7 NFKB1 SOX9'.split(' ')

df = adata_raw_matrix

box_pairs = [("Healthy", "Uninflamed"), 
             ("Healthy", "Inflamed"), 
             ("Uninflamed", "Inflamed")
            ]

order=['Healthy','Uninflamed','Inflamed']

palette=['#2D306E', '#3B735B', '#CF2E1D']

In [ ]:
import seaborn as sns

for i in AMPs_list:
    x = "disease_state" # 横坐标，表型
    y = i   # 纵坐标，基因
    stat_boxplot(df=df, x=x, y=y, box_pairs=box_pairs, order=order, palette=palette)

In [ ]:
adata_raw_matrix_1 = pd.melt(adata_raw_matrix, value_vars=["NR1H4","NR1H3","VDR"],id_vars = ['disease_state'],
             var_name='AMPs', value_name='counts')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from statannot import add_stat_annotation

sns.set_theme(style="ticks",font_scale=1.8)

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(7, 4))
# ax.set_xscale("log")

# Plot the orbital period with horizontal boxes
sns.barplot(x='AMPs',y='counts',hue='disease_state',data=adata_raw_matrix_1,
            capsize=.1,
            errwidth = 1.5,  # 误差棒的粗细
            edgecolor=".2",
            linewidth=1.0,
            hue_order = ["Healthy","Uninflamed","Inflamed"],
            order = ["NR1H4","NR1H3","VDR"],
            palette=["#ffffff","#0d00ff","#ff0000"]
           ) # palette="vlag",字母不需要[]，#开头的颜色需要

# add_stat_annotation(ax, plot='barplot', data=adata_pre_matrix_1, x='AMPs', y='counts', hue='treatment',box_pairs=box_pairs, test='t-test_ind', loc='inside', verbose=2)
# Add in points to show each observation
#sns.stripplot(x='AMPs',y='counts',hue='treatment',data=adata_pre_matrix_1, size=1.5, color=".3", linewidth=0)
#sns.swarmplot(data=adata_pre_matrix_1, x='AMPs',y='counts',hue='treatment')
# Tweak the visual presentation
#ax.xaxis.grid(True)
plt.legend([],[], frameon=False)
sns.despine() # trim=True
ax.set(ylabel="",xlabel="")

In [ ]:
adata_raw_matrix_1 = pd.melt(adata_raw_matrix, value_vars=["LYZ","LCN2","CCL20"],id_vars = ['disease_state'],
             var_name='AMPs', value_name='counts')

import seaborn as sns
import matplotlib.pyplot as plt
from statannot import add_stat_annotation

sns.set_theme(style="ticks",font_scale=1.8)

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(7, 4))
# ax.set_xscale("log")

# Plot the orbital period with horizontal boxes
sns.barplot(x='AMPs',y='counts',hue='disease_state',data=adata_raw_matrix_1,
            capsize=.1,
            errwidth = 1.5,  # 误差棒的粗细
            edgecolor=".2",
            linewidth=1.0,
            hue_order = ["Healthy","Uninflamed","Inflamed"],
            order = ["LYZ","LCN2","CCL20"],
            palette=["#ffffff","#0d00ff","#ff0000"]
           ) # palette="vlag",字母不需要[]，#开头的颜色需要

# add_stat_annotation(ax, plot='barplot', data=adata_pre_matrix_1, x='AMPs', y='counts', hue='treatment',box_pairs=box_pairs, test='t-test_ind', loc='inside', verbose=2)
# Add in points to show each observation
#sns.stripplot(x='AMPs',y='counts',hue='treatment',data=adata_pre_matrix_1, size=1.5, color=".3", linewidth=0)
#sns.swarmplot(data=adata_pre_matrix_1, x='AMPs',y='counts',hue='treatment')
# Tweak the visual presentation
#ax.xaxis.grid(True)
plt.legend([],[], frameon=False)
sns.despine() # trim=True
ax.set(ylabel="",xlabel="")

In [ ]:
adata_pre_matrix_1 = adata_pre_matrix.loc[:,['NR1H4','NR1H3','VDR']]
adata_pre_matrix_1["disease_state"] = adata.obs["disease_state"]

adata_pre_matrix_1 = pd.melt(adata_pre_matrix_1, value_vars=['NR1H4','NR1H3','VDR'],id_vars = ['disease_state'],
             var_name='AMPs', value_name='log2(TP10k+1)')

import seaborn as sns

sns.set(style="ticks", 
        font='Arial', 
        font_scale=1.0, 
        rc={'figure.figsize':(3,6)}
       )

ax = sns.pointplot(x="disease_state", y="log2(TP10k+1)", hue="AMPs",
                   data=adata_pre_matrix_1,
                   markers=["s", "s", "s"],
                   linestyles=["-", "-", "-"],
                   palette = ["#00c3ff","#001aff","#00b359"],
                   order = ["Healthy","Uninflamed","Inflamed"],
                   capsize=.1,     # 误差帽的粗细
                   errwidth = 1.5,  # 误差棒的粗细
                   scale = 1.0     # 连线的簇细，默认值为1
                  )     

## 删除坐标轴标记
ax.set(ylabel="")
ax.set(xlabel="")

# 移除图例
# plt.legend([],[], frameon=False)

# 删除坐标轴上线和右线，当Trim为真时，删除数据数值范围外的坐标轴
sns.despine() # trim=True

print(ax.axis())

# ax.axis([-0.2, 2.2, 0.023832513266243038, 0.6])

# B cells

In [ ]:
adata = sc.read('/mnt/data/Project2021/Gut_immune_surveillance/data/h5ad/B_after_born_harmony_annoation.h5ad')

In [ ]:
adata

In [ ]:
from matplotlib import cm, colors

sc.settings.set_figure_params(dpi=400, figsize = (4, 3), fontsize=12)

colors2 = plt.cm.jet(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

sc.pl.umap(adata, color=['DEFA5','DEFA6','REG3A','PLA2G2A'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['CD79A','IGHA1','REG3G','LYZ'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['NR1H4','NR1H3','VDR','NR1I3'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)
sc.pl.umap(adata, color=['KRT18','CAMP','MUC2','FABP6'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)

In [ ]:
colors2 = plt.cm.Greys_r(np.linspace(0.7,0.8, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,30))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['EPCAM','KRT8'], add_outline=False, frameon=False,size = 1.8, color_map = mymap,)

In [ ]:
sc.settings.set_figure_params(dpi=600, figsize = (4, 3), fontsize=7)
sc.pl.umap(adata, color=['cluster_figure1A'], add_outline=True,
           palette=sc.pl.palettes.vega_20_scanpy, outline_width = (0.2, 0.05), frameon=False, title = '')